# openPMD beamphysics examples

In [ ]:
# Nicer plotting
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams["figure.figsize"] = (8, 4)

# Basic Usage

In [ ]:
from pmd_beamphysics import ParticleGroup

In [ ]:
P = ParticleGroup("data/bmad_particles2.h5")
P

In [ ]:
P.energy

In [ ]:
P["mean_energy"], P.units("mean_energy")

In [ ]:
P.where(P.x < P["mean_x"])

In [ ]:
a = P.plot("x", "px", figsize=(8, 8))

In [ ]:
P.write_elegant("elegant_particles.txt", verbose=True)

# ParticleGroup class

x positions, in meters

In [ ]:
P.x

relativistic gamma, calculated on the fly

In [ ]:
P.gamma

Both are allowed

In [ ]:
len(P), P["n_particle"]

## Basic Statistics

Statistics on any of these. Note that these properly use the .weight array.

In [ ]:
P.avg("gamma"), P.std("p")

Covariance matrix of any list of keys

In [ ]:
P.cov("x", "px", "y", "kinetic_energy")

These can all be accessed with brackets. sigma_ and mean_ are also allowed

In [ ]:
P["sigma_x"], P["sigma_energy"], P["min_y"], P["norm_emit_x"], P["norm_emit_4d"]

Covariance has a special syntax, items separated by __

In [ ]:
P["cov_x__kinetic_energy"]

n-dimensional histogram. This is a wrapper for `numpy.histogramdd`

In [ ]:
H, edges = P.histogramdd("t", "delta_pz", bins=(5, 10))
H.shape, edges

## Slice statistics

ParticleGroup can be sliced along one dimension into chunks of an equal number of particles. Here are the routines to create the raw data.

In [ ]:
ss = P.slice_statistics("norm_emit_x")
ss.keys()

Multiple keys can also be accepted:

In [ ]:
ss = P.slice_statistics("norm_emit_x", "norm_emit_y", "twiss")
ss.keys()

Note that for a slice key `X`, the method will also calculate `mean_X`, `ptp_X`, as `charge` so that a `density` calculated from these. In the special case of `X=t`, the density will be labeled as `current` according to common convention.

## Advanced statisics

Twiss and Dispersion can be calculated.

These are the projected Twiss parameters. 

TODO: normal mode twiss. 

In [ ]:
P.twiss("x")

95% emittance calculation, x and y

In [ ]:
P.twiss("xy", fraction=0.95)

This makes new particles:

In [ ]:
P2 = P.twiss_match(beta=30, alpha=-3, plane="x")
P2.twiss("x")

## Resampling

Particles can be resampled to either scramble the ordering of the particle arrays or subsample.

With no argument or n=0, the same number of particles will be returned:


In [ ]:
P.resample()

With n > 0, particles will be subsampled. Note that this also works for differently weighed particles.

In [ ]:
P.resample(1000)

In [ ]:
P.resample(1000).plot("x", "px", bins=100)

## Units

Units can be retrieved from any computable quantitiy.
These are returned as a pmd_unit type.

In [ ]:
(
    P.units("x"),
    P.units("energy"),
    P.units("norm_emit_x"),
    P.units("cov_x__kinetic_energy"),
    P.units("norm_emit_4d"),
)

In [ ]:
P.units("mean_energy")

In [ ]:
str(P.units("cov_x__kinetic_energy"))

## z vs t

These particles are from Bmad, at the same z and different times

In [ ]:
P.std("z"), P.std("t")

Get the central time:

In [ ]:
t0 = P.avg("t")
t0

Drift all particles to this time. This operates in-place:

In [ ]:
P.drift_to_t(t0)

Now these are at different z, and the same t:

In [ ]:
P.std("z"), P.avg("t"), set(P.t)

## status, weight, id, copy

`status == 1` is alive, otherwise dead. Set the first ten particles to a different status.

`n_alive`, `n_dead` count these

In [ ]:
P.status[0:10] = 0
P.status, P.n_alive, P.n_dead

There is a `.where` convenience routine to make selections easier:

In [ ]:
P0 = P.where(P.status == 0)
P1 = P.where(P.status == 1)
len(P0), P0.charge, P1.charge

Copy is a deep copy:

In [ ]:
P2 = P1.copy()

Charge can also be set. This will re-scale the weight array:

In [ ]:
P2.charge = 9.8765e-12
P1.weight[0:2], P2.weight[0:2], P2.charge

Some codes provide ids for particles. If not, you can assign an id. 

In [ ]:
"id" in P2

This will assign an id if none exists. 

In [ ]:
P2.id, "id" in P2

# Writing

In [ ]:
import h5py
import numpy as np

In [ ]:
newh5file = "particles.h5"

with h5py.File(newh5file, "w") as h5:
    P.write(h5)

with h5py.File(newh5file, "r") as h5:
    P2 = ParticleGroup(h5)

Check if all are the same:

In [ ]:
for key in ["x", "px", "y", "py", "z", "pz", "t", "status", "weight", "id"]:
    same = np.all(P[key] == P2[key])
    print(key, same)

This does the same check:

In [ ]:
P2 == P

Write Astra-style particles

In [ ]:
P.write_astra("astra.dat")

In [ ]:
!head astra.dat

Optionally, a string can be given:

In [ ]:
P.write("particles.h5")

# Plot

Some plotting is included for convenience. See plot_examples.ipynb for better plotting. 

## 1D density plot

In [ ]:
P.plot("x")

## Slice statistic plot

In [ ]:
P.slice_plot("norm_emit_x", "norm_emit_y", ylim=(0, 1e-6))

In [ ]:
P.plot("z", "x")

Any other key that returbs an arrat can be sliced on

In [ ]:
P.slice_plot("sigma_x", slice_key="Jx")

## 2D density plot

In [ ]:
P.plot("x", "px", ellipse=True)

Optionally the figure object can be returned, and the plot further modified.

In [ ]:
fig = P.plot("x", return_figure=True)
ax = fig.axes[0]
ax.set_title("Density Plot")
ax.set_xlim(-50, 50)

## Manual plotting

In [ ]:
import copy

fig, ax = plt.subplots()
ax.set_aspect("equal")
xkey = "x"
ykey = "y"
datx = P[xkey]
daty = P[ykey]
ax.set_xlabel(f"{xkey} ({P.units(xkey)})")
ax.set_ylabel(f"{ykey} ({P.units(ykey)})")

cmap = copy.copy(plt.get_cmap("viridis"))
cmap.set_under("white")
ax.hexbin(datx, daty, gridsize=40, cmap=cmap, vmin=1e-15)

In [ ]:
P.plot("delta_z", "delta_p", figsize=(8, 6))

## Manual binning and plotting

In [ ]:
H, edges = P.histogramdd("delta_z", "delta_p", bins=(150, 150))
extent = [edges[0].min(), edges[0].max(), edges[1].min(), edges[1].max()]

plt.imshow(H.T, origin="lower", extent=extent, aspect="auto", vmin=1e-15, cmap=cmap)

# Multiple ParticleGroup in an HDF5 file

This example has two particlegroups. This also shows how to examine the components, without loading the full data.


In [ ]:
from pmd_beamphysics import particle_paths
from pmd_beamphysics.readers import all_components, component_str

H5FILE = "data/astra_particles.h5"
h5 = h5py.File(H5FILE, "r")

Get the valid paths

In [ ]:
ppaths = particle_paths(h5)
ppaths

Search for all valid components in a single path

In [ ]:
ph5 = h5[ppaths[0]]
all_components(ph5)

Get some info

In [ ]:
for component in all_components(ph5):
    info = component_str(ph5, component)
    print(info)

# Cleanup

In [ ]:
import os

os.remove("astra.dat")
os.remove(newh5file)
os.remove("elegant_particles.txt")